In [1]:
import pandas as pd

In [2]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../Configuration"

In [3]:
import refinitiv.data as rd

In [4]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x225373a8be0 {name='workspace'}>

In [5]:
df_companies = pd.read_csv('Refinitiv_ESG_Universe.csv', delimiter=';')

df_companies.head()

,ISIN,Name
0,DK0010244508,A P MOLLER MAERSK B
1,DE000A1TNNN5,A S CREATION TAPETEN
2,CA0002551095,A&W REVENUE RYLT.INC.FD. UTS.
3,CNE100002RY5,A-LIVING SERVICES 'H'
4,US00181T1079,A-MARK PRECIOUS METALS


In [6]:
df_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9960 entries, 0 to 9959
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISIN    9960 non-null   object
 1   Name    9960 non-null   object
dtypes: object(2)
memory usage: 155.8+ KB


In [7]:
# example, also works with ISIN instead of BASF Ticker
df = rd.get_data(
    universe=["BASFn.DE", "IBM.N"],
    fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
)

df

""


In [9]:
# Stuff that doesn't change with time (Sector/Industry/Country/Region)
# testing the loop
test_frame = df_companies.iloc[:15, :]

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
comp_data_dict = {}

for isin in test_frame["ISIN"]:
    df = rd.get_data(
        universe=isin,
        fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
    )

    comp_data_dict[isin] = df

comp_dfs = []
for key, df in comp_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    comp_dfs.append(df)

# Concatenate the dataframes into a single one
comp_info_df_test = pd.concat(comp_dfs)

# save the new dataframe
comp_info_df_test.to_csv("comp_info_df_test.csv")

comp_info_df_test.head()

,key


In [ ]:
# Stuff that doesn't change with time (Sector/Industry/Country/Region)

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
comp_data_dict = {}

for isin in df_companies["ISIN"]:
    
    df = rd.get_data(
        universe=isin,
        fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
    )

    comp_data_dict[isin] = df

comp_dfs = []
for key, df in comp_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    comp_dfs.append(df)

# Concatenate the dataframes into a single one
comp_info_df = pd.concat(comp_dfs)

# save the new dataframe
comp_info_df.to_csv("comp_info_df.csv")

In [7]:
# testing the loop
test_frame = df_companies.iloc[:15, :]

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
financials_data_dict = {}

for isin in test_frame["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    financials_data_dict[isin] = df

# Assume `dfs_dict` is the dictionary of dataframes
dfs = []
for key, df in financials_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    dfs.append(df)

# Concatenate the dataframes into a single one
result = pd.concat(dfs)

result.to_csv('comp_financials_test.csv')

result.head()

,key


In [ ]:
# loop to collect data for all available companies, stored in a dictionary with a df for each isin
financials_data_dict = {}

for isin in df_companies["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    financials_data_dict[isin] = df

# Assume `dfs_dict` is the dictionary of dataframes
dfs = []
for key, df in financials_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    dfs.append(df)

# Concatenate the dataframes into a single one
result = pd.concat(dfs)

result.to_csv('comp_financials.csv')

result.head()

In [23]:
# Stuff that changes with time (Revenue/Earnings/Employees/ROA/ROE/Free Cash Flow)
# importing the dataframe with the relevant years
#ESG_TR_df = pd.read_csv("tr_esg_df_cleaned.csv")
#
#test_frame = ESG_TR_df.iloc[:30, :]
#
## loop to collect data for all available companies, stored in a dictionary with a df for each isin
#comp_financials_data_dict = {}
#
#for year, isin in zip(test_frame["Year"], test_frame["key"]):
#
#    df = rd.get_history(
#        universe=isin,
#        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
#        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
#        interval="1Y",
#        start=str(year) + "-01-01",
#        end=str(year + 1) + "-01-01",
#    )
#    
#    df.index = pd.to_datetime(df.index)
#    df = df.groupby(df.index.year).first()
#
#    dict_key = isin + str(year)
#    comp_financials_data_dict[dict_key] = df
#
## Assume `dfs_dict` is the dictionary of dataframes
#dfs = []
#for key, df in comp_financials_data_dict.items():
#    df['key'] = key  # Add a new column containing the key
#    dfs.append(df)
#
## Concatenate the dataframes into a single one
#comp_financials_test = pd.concat(dfs)
#
## save the new dataframe
#comp_financials_test.to_csv('comp_financials_test.csv')
#
#comp_financials_test.head()

,Revenue - Mean,Employees - Average,Earnings Per Share - Mean,Return On Assets - Mean,Return On Equity - Mean,Volume,Company Market Cap,Free Cash Flow - Mean,"Weighted Average Cost of Capital, (%)",EBITDA - Mean,Organic Sales Growth - Mean,key
2002,10294556440,,63.19224,,18.19,0,52014960000.0,,,1929737080,,DK00102445082002
2003,26008260480,,109.89529,,17.6,0,182856959999.998993,,,6631310620,,DK00102445082003
2004,30978549110,,179.01468,,20.5845,0,198021780000.0,,,8512477590,,DK00102445082004
2005,32509094730,,188.9474,11.82,19.76857,0,282197520000.0,,,8136143990,,DK00102445082005
2006,47695412750,,132.46312,5.13,12.20555,0,229230540000.0,,,9496474290,,DK00102445082006


In [ ]:
# Stuff that changes with time (Revenue/Earnings/Employees/ROA/ROE/Free Cash Flow)
# importing the dataframe with the relevant years
#ESG_TR_df = pd.read_csv("tr_esg_df_cleaned.csv")
#
#test_frame = ESG_TR_df.iloc[:30, :]
#
## loop to collect data for all available companies, stored in a dictionary with a df for each isin
#comp_financials_data_dict = {}
#
#def financials_collection(year, isin):
#
#
#
#
#    df = rd.get_history(
#        universe=isin,
#        fields=["TR.Revenue", "TR.Earnings", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
#        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
#        interval="1Y",
#        start=str(year) + "-01-01",
#        end=str(year + 1) + "-01-01",
#    )
#    
#    df.index = pd.to_datetime(df.index)
#    df = df.groupby(df.index.year).first()
#
#    dict_key = isin + str(year)
#
#    comp_financials_data_dict[isin] = df
#
## Assume `dfs_dict` is the dictionary of dataframes
#dfs = []
#for key, df in comp_financials_data_dict.items():
#    df['key'] = key  # Add a new column containing the key
#    dfs.append(df)
#
## Concatenate the dataframes into a single one
#comp_financials_test = pd.concat(dfs)
#
## save the new dataframe
#comp_financials_test.to_csv('comp_financials_test.csv')
#
#comp_financials_test.head()

In [8]:
rd.close_session()